In [1]:
import mxnet as mx
import gluonnlp as nlp

ctx = mx.gpu()

In [2]:
# TextCNN Components
# 1D Convolution
conv1d = mx.gluon.nn.Conv1D(4, 5, activation='relu')

# Max-Over-Time Pooling
max_over_time_pooling = mx.gluon.nn.GlobalMaxPool1D()

In [3]:
conv1d.collect_params().initialize()
output = conv1d(mx.nd.array([[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]]))
print(output)

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/mxnet_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-08-05 11:04:54.697 ip-172-31-28-47:21485 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-08-05 11:04:54.728 ip-172-31-28-47:21485 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.

[[[0.38793308 0.5004758  0.6130185  0.72556126 0.838104   0.95064676]
  [0.2522667  0.36388856 0.47551042 0.58713233 0.6987542  0.81037605]
  [0.12269916 0.14470303 0.16670692 0.18871081 0.21071473 0.23271859]
  [0.         0.         0.         0.         0.         0.        ]]]
<NDArray 1x4x6 @cpu(0)>


In [4]:
max_over_time_pooling(output)


[[[0.95064676]
  [0.81037605]
  [0.23271859]
  [0.        ]]]
<NDArray 1x4x1 @cpu(0)>

In [5]:
# RNNs MXNet Implementation Example
class RNNModel(mx.gluon.Block):
    """
    A basic RNN Model
    """
    def __init__(self, num_hidden, num_layers, embed_size, **kwargs):
        
        super(RNNModel, self).__init__(**kwargs)
        
        self.rnn = mx.gluon.rnn.RNN(
            num_hidden,
            num_layers,
            input_size=embed_size)
        
        
    def forward(self, inputs, hidden):
        output, hidden = self.rnn(inputs, hidden)
        return output, hidden

In [6]:
# RNN with 3 hidden cells, 1 layer and expecting inputs with 20 embeddings
rnn = RNNModel(3, 1, 20)
rnn.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [7]:
# Initial Hidden Values
hidden_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

In [8]:
# Inputs generation
# Let's input 3 words 
# Each word is passed through a 20-component embedding model
inputs = []

# Each word representation
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))

In [9]:
rnn_hidden = hidden_initial
outputs = []

for index in range(3):
    rnn_output, rnn_hidden = rnn(inputs[index], rnn_hidden)
    outputs.append(rnn_output)

print(rnn_output, rnn_hidden)


[[[0.         0.         0.08548212]]

 [[1.1657329  0.40813452 0.18241143]]

 [[0.         0.         0.        ]]]
<NDArray 3x1x3 @gpu(0)> [
[[[0. 0. 0.]]]
<NDArray 1x1x3 @gpu(0)>]


In [10]:
# LSTMs MXNet Implementation Example
class LSTMModel(mx.gluon.Block):
    """
    A basic LSTM Model
    """
    def __init__(self, num_hidden, num_layers, embed_size, **kwargs):
        
        super(LSTMModel, self).__init__(**kwargs)
        
        self.lstm = mx.gluon.rnn.LSTM(
            num_hidden,
            num_layers,
            input_size=embed_size)
        
        
    def forward(self, inputs, hidden):
        output, hidden = self.lstm(inputs, hidden)
        return output, hidden

In [11]:
# LSTM with 3 hidden cells, 1 layer and expecting inputs with 20 embeddings
lstm = LSTMModel(3, 1, 20)
lstm.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [12]:
# Initial Hidden Values
hidden_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

# Initial State Values
state_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

In [13]:
lstm_hidden = [hidden_initial, state_initial]
outputs = []

for index in range(3):
    lstm_output, lstm_hidden = lstm(inputs[index], lstm_hidden)
    outputs.append(lstm_output)

print(lstm_output, lstm_hidden)


[[[-0.10296986 -0.03880391 -0.11327321]]

 [[-0.31936553  0.08149263 -0.19445275]]

 [[-0.06096089  0.04764624  0.01065951]]]
<NDArray 3x1x3 @gpu(0)> [
[[[-0.06096089  0.04764624  0.01065951]]]
<NDArray 1x1x3 @gpu(0)>, 
[[[-0.3053535   0.07585216  0.01485998]]]
<NDArray 1x1x3 @gpu(0)>]


In [14]:
# Transformers MXNet Implementation Example
# Transformer with 6 layers (encoder and decoder), 2 parallel heads, and expecting inputs with 20 embeddings
transformer_encoder, transformer_decoder, _ = nlp.model.transformer.get_transformer_encoder_decoder(
    num_layers=6,
    num_heads=2,
    units=20)

transformer_encoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
transformer_decoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [15]:
# Only encoder shown
encoded_inputs, _ = transformer_encoder(inputs[0])

print(encoded_inputs)


[[[-1.8816305  -1.2331309   0.24438679  0.6218342   0.16494197
   -1.3730463   0.5399733   0.05484034  0.61006564  0.4691743
    2.779219   -0.2504159   1.1693825  -0.9574291   0.15646233
   -0.11039922 -0.17834105 -0.9232324  -0.54143316  0.6387784 ]]

 [[ 1.3105048   0.56982553 -0.3358584  -1.097387    0.70062995
   -1.004579   -0.75051713 -0.60301656 -1.3017621  -1.4431249
   -0.50571865  0.22127484  1.2398077   0.51822484 -0.19751823
    1.5655922  -1.5669955   1.4241285   1.1587926   0.09769692]]

 [[ 1.0646535   0.13031901 -1.1666297   0.05262218  0.59379524
   -0.28079516  0.21707663  0.02480992 -0.4232612   1.5293657
   -0.52167237 -1.9738344   2.0619779  -0.26011172 -1.7410742
    1.0688411   0.18393981  0.55884945  0.09433853 -1.21321   ]]]
<NDArray 3x1x20 @gpu(0)>
